In [1]:
#解析 xml 之例外處理
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [2]:
#import 策略程式碼
import Ipynb_importer
import KD9
import Cont_3
import Gold_X_Death
import autoRemindPrice

importing Jupyter notebook from KD9.ipynb
importing Jupyter notebook from Cont_3.ipynb
importing Jupyter notebook from Gold_X_Death.ipynb
importing Jupyter notebook from autoRemindPrice.ipynb


In [3]:
# basic
import numpy as np
import pandas as pd
import requests
import re
import io
from bs4 import BeautifulSoup
import pymongo
# visual
import matplotlib.pyplot as plt
%matplotlib inline
#time
import datetime 
#talib
import talib
from talib import abstract
#gui
import tkinter as tk
from  tkinter import ttk  
from tkinter import messagebox 
import tkinter.font as tkFont

In [4]:
from tkinter.ttk import *

import matplotlib
matplotlib.use("TkAgg")

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [5]:
#爬蟲 yahoo csv
def crawl_price(stock_id):
    now = int(datetime.datetime.now().timestamp())+86400
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=0&period2=" + str(now) + "&interval=1d&events=history&crumb=hP2rOschxO0"
    response = requests.post(url)
    f = io.StringIO(response.text)
    df = pd.read_csv(f, index_col='Date', parse_dates=['Date'] )
    return df

In [6]:
#爬取 api 並解析
def get_webservice(url):
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content)
    r = soup.find('string').text
    return r

## 第一種

In [36]:
#9日 K值低於 20% 買進，高於 80% 賣出
#9日 K值低於 20% 買進，高於 80% 賣出
def testKD92080set(start_money,close,data,sProfit,sLoss):
    K = data.iloc[0:,1] #取 K 值
    #D = data.iloc[0:,2] #取 D 值
    #DIF = data2.iloc[0:,0] #取 DIF 值
    #DEA = data2.iloc[0:,1] #取 DEA 值
    #MACD = data2.iloc[0:,2] #取 MACD 值
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    principal = float(start_money)
    stock_count = 0 #股票張數
    bought_money = 0 #剩餘金額
    final_money = 0 #最終金額
    profit = 0 #獲利
    rateOfReturn = 0.0 #報酬率
    record = [] #存取購買股票時的股價
    lastROP = [] #存取報酬率

    try:
        
        for i in range(len(K)):
            print(date[i])
            print(i)
            print(close[i])
            if(stock_count==0):
                if(float(start_money)>=close[i]*1000):
                    if(K[i]<20):
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        stock_count = stock_count + 1
                        start_money = float(start_money) - close[i]*1000
                        record.append(close[i]) #將買的股價放入recorde陣列裡
                        print(record)
                        print('剩餘資金為:',start_money)
                        print('擁有'+str(stock_count)+ '張股票')
                        table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("尚未達到買點")
                        #continue
                else:
                    print("錢不購買")
                    #continue
            else:
                if(close[i] >= record[-1]+float(sProfit)): #現在的股價 >= 停利條件
                    print('現在的股價=',close[i])
                    print('條件: 現股>=最後一次購買股價+10 = ') # recode[-1]為最後一次交易的股價
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價+5',record[-1]+float(sProfit))
                    print('已達停利標準')
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(close[i] <= record[-1]-float(sLoss)):
                    print('現在的股價=',close[i]) 
                    print('條件: 現股<=最後一次購買股價-5 = ')
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價-5',record[-1]-float(sLoss))
                    print('已達停損標準')
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(K[i]<20):
                    if(float(start_money)>=close[i]*1000):
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        stock_count = stock_count + 1
                        start_money = float(start_money) - close[i]*1000
                        record.append(close[i]) #將買的股價放入recorde陣列裡
                        print(record)
                        print('剩餘資金為:',start_money)
                        print('擁有'+str(stock_count)+ '張股票')
                        table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("錢不夠再買一張股票")
                        #continue
                elif(K[i]>80):
                    print('K值為:',K[i])
                    print("手上有"+str(stock_count)+"張股票可賣")
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money
                else:
                    print("尚未達到買賣點")
                    #continue
        print("\n")          
        
        #textPanel.insert("end", rateOfReturn)
        textPanel1.insert("end", lastROP[-1])
        
        figure = Figure(figsize=(6, 3), dpi=100)
        ax1 = figure.add_subplot(1, 1, 1)
        #ax1.plot(datelist,closelist,color= 'r',linewidth=2,linestyle="-",label='Price')

       # ax1.legend(loc = 'upper left', frameon=True)
        #ax2 = ax1.twinx()
        #ax2.plot(datelist,closelist,color= 'r',linewidth=2,linestyle="-",label='Price')

        #ax2.legend(loc = 'upper right', frameon=True)
        canvas = FigureCanvasTkAgg(figure, frm_c2)
        canvas.get_tk_widget().grid(row=3, column=0)  
        
        print('總報酬率',lastROP[-1])
        return lastROP[-1]
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

## 第二種

In [27]:
#K值連續三天低於20後反彈(買進)，K值連續三天高於80後反彈(賣出)
def testKDcontSet(start_money,close,data,sProfit,sLoss):
    K = data.iloc[0:,1] #取 K 值
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    principal = float(start_money)
    stock_count = 0 #股票張數
    #bought_money = 0 #剩餘金額
    final_money = 0 #最終金額
    profit = 0 #獲利
    rateOfReturn = 0.0 #報酬率
    record = [] #存取購買股票時的股價
    lastROP = [] #存取報酬率
    datelist=[]
    closelist=[]
    print("2",start_money)
    try:
        for i in range(3,len(K)):
            if stock_count == 0:
                if float(start_money)>=close[i]*1000:
                    if K[i-1]<20 and K[i-2]<20 and K[i-3]<20: 
                        if K[i]>20: # K值連續 3 天小於 20 後上升
                            stock_count = stock_count + 1 #買一張股票
                            start_money = float(start_money) - (float(close[i])*1000) #剩餘資金 = 初始資金 - 當日股價
                            print(date[i-3],'K值為:',K[i-3])
                            print(date[i-2],'K值為:',K[i-2])
                            print(date[i-1],'K值為:',K[i-1])
                            print(date[i],'K值為:',K[i])
                            print("買了一張股票，股價為:",close[i])
                            record.append(close[i]) #將買的股價放入record陣列裡
                            print(record)
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                    else:
                        print("未達買點")
                        continue
                else:
                    print("資金不足")
                    continue

            else:
                if(close[i] >= record[-1]+float(sProfit)): #現在的股價 >= 停利條件
                        datelist.append(date[i])
                        closelist.append(close[i])
                        print(date[i])
                        print('現在的股價=',close[i]) 
                        print('條件: 現股>=最後一次購買股價+5 = ') # recode[-1]為最後一次交易的股價
                        print('最後一次購買股價',record[-1])
                        print('最後一次購買股價+5',record[-1]+float(sProfit))
                        print('已達停利標準')
                        final_money = start_money + stock_count*close[i]*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:] = []
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                        #lastROP.append(rateOfReturn)
                        start_money = final_money #將初始金額改為賣出後的總金額
                        continue
                elif(close[i] <= record[-1]-float(sLoss)): #現在的股價 <= 停利條件
                        datelist.append(date[i])
                        closelist.append(close[i])
                        print(date[i])
                        print('現在的股價=',close[i]) 
                        print('條件: 現股<=最後一次購買股價-5 = ')
                        print('最後一次購買股價',record[-1])
                        print('最後一次購買股價-5',record[-1]-float(sLoss))
                        print('已達停損標準')
                        final_money = start_money + stock_count*close[i]*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:] = []
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                        #lastROP.append(rateOfReturn)
                        start_money = final_money #將初始金額改為賣出後的總金額
                        continue

                elif K[i-1]<20 and K[i-2]<20 and K[i-3]<20:
                    if K[i]>20:
                        if float(start_money)>=close[i]*1000:
                            stock_count = stock_count + 1 #買一張股票
                            start_money = float(start_money) - (float(close[i])*1000) #剩餘資金 = 初始資金 - 當日股價
                            print(date[i-1],'K值為:',K[i-1])
                            print(date[i-2],'K值為:',K[i-2])
                            print(date[i-3],'K值為:',K[i-3])
                            print(date[i],'K值為:',K[i])
                            print("買了一張股票，股價為:",close[i])
                            record.append(close[i]) #將買的股價放入record陣列裡
                            print(record)
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        else:
                            print("資金不足")
                elif K[i-1]>80 and K[i-2]>80 and K[i-3]>80:
                    if K[i]<80:
                        datelist.append(date[i])
                        closelist.append(close[i])
                        print(date[i-1],'K值為:',K[i-1])
                        print(date[i-2],'K值為:',K[i-2])
                        print(date[i-3],'K值為:',K[i-3])
                        print(date[i],'K值為:',K[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:]=[]
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , )
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit))
                        #lastROP.append(rateOfReturn)
                        start_money = final_money
                else:
                    print(date[i])
                    print("未達買賣點")
        print("\n")        
        #textPanel.insert("end", lastROP[-1])
        textPanel2.insert("end", lastROP[-1])
        
        figure = Figure(figsize=(6, 3), dpi=100)
        ax1 = figure.add_subplot(1, 1, 1)
        #ax1.plot(datelist,lastROP,color= 'g',linewidth=2,linestyle="-",label='ROP')
        #ax1.set_ylim(-100000,300000)
        #ax1.legend(loc = 'upper left', frameon=True)
        ax2 = ax1.twinx()
        ax2.plot(datelist,closelist,color= 'r',linewidth=2,linestyle="-",label='Price')
        #ax2.set_ylim(0,400)
        #ax2.yticks(np.linespace(100, 5000, 10))
        ax2.legend(loc = 'upper right', frameon=True)
        canvas = FigureCanvasTkAgg(figure, frm_c2)
        canvas.get_tk_widget().grid(row=3, column=0)
        
        print('總報酬率',lastROP[-1])
        return lastROP[-1] 
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

## 第三種

In [28]:
## K值向上突破D值 (黃金交叉)，K值向下突破 D值 (死亡交叉)
def testKDcrossSet(start_money,close,data,sProfit,sLoss):
    K = data.iloc[0:,1] #取 K 值
    D = data.iloc[0:,2] #取 D 值
    principal = float(start_money)
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    stock_count = 0 #股票張數
    bought_money = 0.0 #剩餘金額
    final_money = 0.0 #最終金額
    profit = 0.0 #獲利
    rateOfReturn = 0.0 #報酬率
    lastROP = []  
    record = []
    datelist = []
    closelist=[]
    try:  
        for i in range(1,len(K)):
            #print(date[i])
            print(date[i])
            print(i)
            print(close[i])
            if(len(record)==0): #手上沒股票
                if(float(start_money)>=close[i-1]*1000): #身上錢夠
                    if (K[i-1]<D[i-1] and K[i]>D[i]): #黃金交叉，買股票
                            #datelist.append(date[i-1])
                            stock_count = stock_count + 1
                            start_money = float(start_money) - (float(close[i-1])*1000)
                            record.append(close[i-1])
                            print("record:",record)
                            print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                            print("買了一張股票，股價為:",close[i-1])
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i-1],close[i-1],str(stock_count),start_money))
                    else: #不是黃金交叉，直接進入下個迴圈
                        continue
                else: #錢不夠，進入下個迴圈
                    continue
            elif(len(record)!=0): #身上有股票，判斷是否符合停利/停損，是->賣掉，否->判斷為黃金交叉(買)/死亡交叉(賣)/都不是(繼續)
                    if close[i-1] >= record[-1]+float(sProfit) :#停利，賣股票
                        datelist.append(date[i-1])
                        closelist.append(close[i-1])
                        print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        rateOfReturn = round(rateOfReturn, 2)
                        lastROP.append(rateOfReturn)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money
                    elif close[i-1] <= record[-1]-float(sLoss) :
                        datelist.append(date[i-1])
                        closelist.append(close[i-1])
                        print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        rateOfReturn = round(rateOfReturn, 2)
                        lastROP.append(rateOfReturn)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money
                    elif (K[i-1]<D[i-1] and K[i]>D[i]):#買
                        #datelist.append(date[i-1])
                        if float(start_money)>=close[i]*1000:
                            stock_count = stock_count + 1 #買一張股票
                            bought_money = start_money - (float(close[i-1])*1000) #剩餘資金 = 上一次交易剩餘資金 - 當日股價
                            record.append(close[i])
                            print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                            print("買了一張股票，股價為:",close[i-1])
                            print('剩餘資金為:',bought_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i-1],close[i-1],str(stock_count),bought_money))
                            start_money = bought_money
                        else:
                            continue
                    elif (K[i-1]>D[i-1] and K[i]<D[i]):
                        datelist.append(date[i-1])
                        closelist.append(close[i-1])
                        print(date[i],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        rateOfReturn = round(rateOfReturn, 2)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        lastROP.append(rateOfReturn)
                        print(lastROP)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money

        textPanel3.insert("end",lastROP[-1])
        figure = Figure(figsize=(6, 3), dpi=100)
        ax1 = figure.add_subplot(1, 1, 1)
        #ax1.plot(datelist,lastROP,color= 'g',linewidth=2,linestyle="-",label='ROP')
        #ax1.set_ylim(-100000,300000)
        #ax1.legend(loc = 'upper left', frameon=True)
        ax2 = ax1.twinx()
        ax2.plot(datelist,closelist,color= 'r',linewidth=2,linestyle="-",label='Price')
        #ax2.set_ylim(0,400)
        #ax2.yticks(np.linespace(100, 5000, 10))
        ax2.legend(loc = 'upper right', frameon=True)
        canvas = FigureCanvasTkAgg(figure, frm_c2)
        canvas.get_tk_widget().grid(row=3, column=0)

        
        return lastROP[-1]
    
        
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

In [ ]:
def first():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    sProfit  = sProfit_entry.get()
    sLoss = sLoss_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close,2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
    #data.plot()
    data.reset_index(inplace=True)
    
    return  testKD92080set(start_money,close2,data,sProfit,sLoss)


In [ ]:
def second():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    sProfit  = sProfit_entry.get()
    sLoss = sLoss_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close,2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
    #data.plot()
    
    data.reset_index(inplace=True)
        
    return testKDcontSet(start_money,close2,data,sProfit,sLoss)


In [ ]:
def third():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    sProfit  = sProfit_entry.get()
    sLoss = sLoss_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close,2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
    #data.plot()
    #figure = Figure(figsize=(6, 3), dpi=100)
    #plot = figure.add_subplot(1, 1, 1)
    #plot.plot(data)
    #canvas = FigureCanvasTkAgg(figure, frm_r)
    #canvas.get_tk_widget().grid(row=1, column=0)
    
    data.reset_index(inplace=True)
    
    return testKDcrossSet(start_money,close2,data,sProfit,sLoss)

In [ ]:
def callfunc2():
    ans = v.get()
    if(ans=="K值小於設定值進場，大於設定值出場"):
        stockid = stockCode_entry.get()
        sProfit = sProfit_entry.get()
        sLoss = sLoss_entry.get()
        KD9.start(stockid,sProfit,sLoss)
    elif(ans=="K值連續3天小於設定值後反彈進場"):
        stockid = stockCode_entry.get()
        sProfit = sProfit_entry.get()
        sLoss = sLoss_entry.get()
        Cont_3.start(stockid,sProfit,sLoss)
    elif(ans=="黃金交叉與死亡交叉"):
        stockid = stockCode_entry.get()
        sProfit = sProfit_entry.get()
        sLoss = sLoss_entry.get()
        Gold_X_Death.start(stockid,sProfit,sLoss)
def callfunc():
    ans = v.get()
    if(ans=="K值小於設定值進場，大於設定值出場"):
        first()
    elif(ans=="K值連續3天小於設定值後反彈進場"):
        second()
    elif(ans=="黃金交叉與死亡交叉"):
        third()
def Run3():
    a = int(first())
    b = int(second())
    c = int(third())
    if a>b:
        if b>c:
            print("第一個策略獲利最多")
            bestanswer = "第一個策略獲利最多"
        else:
            if a>c:
                print('第一個策略獲利最多')
                bestanswer = "第一個策略獲利最多"

            else:
                print('第三個策略獲利最多')
                bestanswer = "第三個策略獲利最多"
    elif a<b:
        if b<c:
            print('第三個策略獲利最多')
            bestanswer = "第三個策略獲利最多"
        else:
            if c>a:
                print('第二個策略獲利最多')
                bestanswer = "第二個策略獲利最多"
            else:
                print('第二個策略獲利最多')
                bestanswer = "第二個策略獲利最多"
    answer2 = tk.Label(frm_l, text= bestanswer)
    answer2.grid(row =18, column = 1,sticky='W')
def deletetable():
    x = table.get_children()
    for item in x: 
        table.delete(item) 
    #textPanel.delete('1.0','end') 
    textPanel1.delete('1.0','end') 
    textPanel2.delete('1.0','end') 
    textPanel3.delete('1.0','end')

In [ ]:
def order():
    win.destroy()
    order = tk.Tk()
    order.title('下單')
    order.geometry("1100x600")
    frame = tk.Frame(order)
    frame.grid(row = 0)
    frame_l = tk.Frame(frame, ) #左視窗
    frame_r = tk.Frame(frame) #右視窗
    frame_c = tk.Frame(frame,)
    frame_l.grid(row = 1,column = 0)
    frame_r.grid(row = 1,column = 2)
    frame_c.grid(row = 1,column = 1)
    
    ft1 = tkFont.Font(family='Fixdsys', size=16, weight=tkFont.BOLD)
    
    
    
    
    def getOrderRecorde(): #委託紀錄id(

        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("委託時間","委託單ID","股票代碼","公司名稱","委託價格","委託數量","買賣類型","掛單類型")
        #表示列,不顯示
        table.column("委託時間",width=160)
        table.column("委託單ID",width=100)
        table.column("股票代碼",width=80)
        table.column("公司名稱",width=80)
        table.column("委託價格",width=80)
        table.column("委託數量",width=80)
        table.column("買賣類型",width=80)
        table.column("掛單類型",width=80)

        #顯示標題
        table.heading("委託時間",text="委託時間")
        table.heading("委託單ID",text="委託單ID")
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("委託價格",text="委託價格")
        table.heading("委託數量",text="委託數量")
        table.heading("買賣類型",text="買賣類型")
        table.heading("掛單類型",text="掛單類型")
        table.grid(row = 0, column = 0)
        
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryWaitingOrderListGVE3XML_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&Language=TC"
        client = get_webservice(url)
        root2 = ET.fromstring(client)
        i = 1
        for country in root2.findall('Order'):
            print("第",i,"筆委託紀錄")
            OrderTime = country.get('OrderTime')
            OrderID = country.get('OrderID')
            AssetID = country.get('AssetID')
            CompName = country.get('CompName')
            Price = country.get('Price')
            Volume = country.get('Volume')
            BSAction = country.get('BSAction')
            OrderType = country.get('OrderType')

            print(OrderTime, OrderID, AssetID, CompName, Price, Volume, BSAction, OrderType)
            i+=1
            table.insert("","end",values=(OrderTime,OrderID,AssetID,CompName,Price,Volume,BSAction,OrderType))
            
        def deletetable(): #清除 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
    def getInstock(): #庫存
        table=ttk.Treeview(frame_r,show='headings') #表格
        table["columns"]=("股票代碼","公司名稱","持有數量","持有成本","現價","漲跌","盈虧","累積未實現損益")
        #表示列,不顯示
        table.column("股票代碼",width=160)
        table.column("公司名稱",width=80)
        table.column("持有數量",width=80)
        table.column("持有成本",width=80)
        table.column("現價",width=80)
        table.column("漲跌",width=80)
        table.column("盈虧",width=80)
        table.column("累積未實現損益",width=80)

        #顯示標題
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("持有數量",text="持有數量")
        table.heading("持有成本",text="持有成本")
        table.heading("現價",text="現價")
        table.heading("漲跌",text="漲跌")
        table.heading("盈虧",text="盈虧")
        table.heading("累積未實現損益",text="累積未實現損益")
        table.grid(row = 0, column = 0)
        
        url = "http://61.220.30.176/WebOrder/GVEAccount.asmx/QueryTodayPositionGve3XML_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&Language=TC&SubTotalItem=&SortItem=AssetCode%20Asc"
        client = get_webservice(url)
        root1 = ET.fromstring(client)
        i = 1
        for country in root1.findall('PortfolioAsset'):
            print("第",i,"筆庫存紀錄")
            AssetCode = country.get('AssetCode')
            CompName = country.get('CompName')
            Hold = country.get('Hold')
            Cost = country.get('Cost')
            QuotePrice = country.get('QuotePrice')
            UpDown = country.get('UpDown')
            PL = country.get('PL')
            UnRealizedPL = country.get('UnRealizedPL')
            print(AssetCode, CompName, Hold, Cost, QuotePrice, UpDown, PL, UnRealizedPL)
            i+=1
            table.insert("","end",values=(AssetCode,CompName,Hold,Cost,QuotePrice,UpDown,PL,UnRealizedPL))
        def deletetable(): #清除 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
    def getDeal(): #成交紀錄
        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("交易時間","股票代碼","公司名稱","買賣類型","掛單類型","成交價","成交量","手續費","交易訊息")
        table.column("交易時間",width=160)   
        table.column("股票代碼",width=80)
        table.column("公司名稱",width=80)
        table.column("買賣類型",width=80)
        table.column("掛單類型",width=80)
        table.column("成交價",width=80)
        table.column("成交量",width=80)
        table.column("手續費",width=80)
        table.column("交易訊息",width=80)

        table.heading("交易時間",text="交易時間")  
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("買賣類型",text="買賣類型")
        table.heading("掛單類型",text="掛單類型")
        table.heading("成交價",text="成交價")
        table.heading("成交量",text="成交量")
        table.heading("手續費",text="手續費")
        table.heading("交易訊息",text="交易訊息")
        table.grid(row = 0, column = 0)
        
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryDealLogGVE3ByGMRDayRangeLiteXML_NS?GMRID=1111708496&StartDate=2019/01/02&EndDate=2019/12/20&Language=TC"
        client = get_webservice(url)
        root = ET.fromstring(client)
        i = 1
        for country in root.findall('GVEUserLog'):
            print("第",i,"筆成交紀錄")
            #LogID = country.get('LogID')
            LogTime = country.get('LogTime')
            AssetCode = country.get('AssetCode')
            CompName = country.get('CompName')
            BSAction = country.get('BSAction')
            OrderType = country.get('OrderType')
            Price = country.get('Price')
            Volume = country.get('Volume')
            fee = country.get('fee')
            logdesc = country.get('logdesc')
            print(LogTime, AssetCode, CompName, BSAction, OrderType, Price, Volume, fee, logdesc)
            i+=1
            table.insert("","end",values=(LogTime,AssetCode,CompName,BSAction,OrderType,Price,Volume,fee,logdesc))
            
        def deletetable(): #清除
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
    def getOrder(): #下單
    
            compcode = CompCode_Entry.get()
            price = Price_Entry.get()
            volume = Volume_Entry.get()
            bsaction = BSAction_Entry.get()
            ordertype = OrderType_Entry.get()

            url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/PutOrderXML3?GMRIDStr=BCSP&CompCode="+compcode+"&Price="+price+"&Volume="+volume+"&BSAction="+bsaction+"&OrderType="+ordertype+"&IsOddLot=0&Currency=TWD&OrderNote=ROD&OCType=0&CombineNo=&OrderParameter=0&Lang=TC&str_ip=127.0.0.1"
            client = get_webservice(url)
            new = client.split("\"")
            if new[1] == "Success":
                tk.messagebox.showinfo(title='回報', message='Success')
            elif new[1] == "Failure":
                tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:'+new[5])
            else:
                tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:格式輸入錯誤')

    def getPrice():
        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("成交價","成交量","委買價","委買量","委賣價","委賣量","昨收價","漲停價","跌停價")
         #表示列,不顯示
        table.column("成交價",width=160)
        table.column("成交量",width=80)
        table.column("委買價",width=80)
        table.column("委買量",width=80)
        table.column("委賣價",width=80)
        table.column("委賣量",width=80)
        table.column("昨收價",width=80)
        table.column("漲停價",width=80)
        table.column("跌停價",width=80)
        #顯示標題
        table.heading("成交價",text="成交價")  
        table.heading("成交量",text="成交量")
        table.heading("委買價",text="委買價")
        table.heading("委買量",text="委買量")
        table.heading("委賣價",text="委賣價")
        table.heading("委賣量",text="委賣量")
        table.heading("昨收價",text="昨收價")
        table.heading("漲停價",text="漲停價")
        table.heading("跌停價",text="跌停價")
        table.grid(row = 0, column = 0)
        
        compcode = CompCode_Entry.get()
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryQuote5Price?compcode=" + compcode
        client = get_webservice(url)
        r = ET.fromstring(client)
        r_new = "您查詢的股價為:\n"+"成交價:"+str(r[0][0].text)+\
                                "\n"+"成交量:"+str(r[0][1].text)+\
                                "\n"+"委買價:"+str(r[0][2].text)+\
                                "\n"+"委買量:"+str(r[0][22].text)+\
                                "\n"+"委賣價:"+str(r[0][12].text)+\
                                "\n"+"委賣量:"+str(r[0][23].text)+\
                                "\n"+"昨收價:"+str(r[0][24].text)+\
                                "\n"+"漲停價:"+str(r[0][25].text)+\
                                "\n"+"跌停價:"+str(r[0][26].text)+"\n"
        
        
        table.insert("","end", values=(str(r[0][0].text),str(r[0][1].text),str(r[0][2].text),str(r[0][22].text),str(r[0][12].text),str(r[0][23].text),str(r[0][24].text),str(r[0][25].text),str(r[0][26].text)))
        def deletetable(): #清除 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
        
    def cancelOrder():
        cancle_id = cancle_entry.get()
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/CancelOrderStr_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&OrderID=" + cancle_id
        client = get_webservice(url)
        getOrderRecorde()

    #Label
    title = tk.Label(frame_l, text = "委託下單",font=ft1)
    title.grid(row=0,column=0)
    title = tk.Label(frame_l, text = "查詢功能",font=ft1)
    title.grid(row=7,column=0)
    CompCode = tk.Label(frame_l, text="股票代碼")#股票代碼
    CompCode.grid(row=1,column=0)
    Price = tk.Label(frame_l, text="委託價格")#價格
    Price.grid(row=2,column=0)
    Volumn = tk.Label(frame_l, text="委託數量")#成交量
    Volumn.grid(row=3,column=0)
    BSAction = tk.Label(frame_l, text="買賣類型")#使用代碼 普通買進(B),普通賣出(S),融資買入(MB),融資賣出(RB),融券買入(RS)
    BSAction.grid(row=4,column=0)
    OrderType = tk.Label(frame_l, text="下單類型")#下單類型 漲停價(LMTU),跌停價(LMTD),限價單(LMT)，期權市價單：MKT
    OrderType.grid(row=5,column=0)
    cancleOrder = tk.Label(frame_l, text = "取消委託",font=ft1)#取消委託
    cancleOrder.grid(row=14,column=0)
    cancle = tk.Label(frame_l, text = "輸入ID:")
    cancle.grid(row = 15, column = 0)
    
    #預設輸入
    GMRIDstr = tk.StringVar(value="BCSP")
    CompCode = tk.StringVar(value="2330.tw")
    Volume = tk.StringVar(value="1000")
    BSAction = tk.StringVar(value="B")
    OrderType = tk.StringVar(value="MKT")

    #輸入框
    CompCode_Entry = tk.Entry(frame_l,textvariable=CompCode)
    CompCode_Entry.grid(row=1,column=1)
    Price_Entry = tk.Entry(frame_l,textvariable=Price)
    Price_Entry.grid(row=2,column=1)
    Volume_Entry = tk.Entry(frame_l,textvariable=Volume)
    Volume_Entry.grid(row=3,column=1)
    BSAction_Entry = tk.Entry(frame_l,textvariable=BSAction)
    BSAction_Entry.grid(row=4,column=1)
    OrderType_Entry = tk.Entry(frame_l,textvariable=OrderType)
    OrderType_Entry.grid(row=5,column=1)
    cancle_entry = tk.Entry(frame_l)
    cancle_entry.grid(row=15,column = 1)
#-------------------------Button-------------------------#    
    order_btn = tk.Button(frame_l , text="下單", command=getOrder) 
    order_btn.grid(row = 3,column = 2,padx = 10)    
    record_btn = tk.Button(frame_l, text="委託紀錄", command=getOrderRecorde)
    record_btn.grid(row = 8,column = 0,pady = 10)
    instock_btn = tk.Button(frame_l, text="庫存紀錄", command=getInstock)
    instock_btn.grid(row = 8,column = 1,pady = 10)
    deal_btn = tk.Button(frame_l, text="成交紀錄", command = getDeal)
    deal_btn.grid(row = 9,column = 0,pady = 10)
    stockprice_btn = tk.Button(frame_l, text="即時股價", command=getPrice)
    stockprice_btn.grid(row = 9,column = 1,pady = 10)
    cancle_btn = tk.Button(frame_l, text="取消委託", command = cancelOrder)
    cancle_btn.grid(row = 15,column = 2,padx = 10)


In [37]:
win = tk.Tk()#建立主視窗
win.title("股票回測")#視窗標題
win.geometry("1050x768")#寬*高
win.resizable(True, True)#限制縮放
#框架
frm = tk.Frame(win)
frm.grid(row = 0)
frm_l = tk.Frame(frm, ) #左視窗
frm_r = tk.Frame(frm,) #右視窗
frm_c = tk.Frame(frm,) #右下視窗
frm_k = tk.Frame(frm,) #左下視窗
frm_c2 = tk.Frame(frm,)
frm_k2 = tk.Frame(frm,)
frm_k3 = tk.Frame(frm,)
frm_k4 = tk.Frame(frm,)
frm_k5 = tk.Frame(frm,)
frm_k6 = tk.Frame(frm)
frm_k.grid(row = 2,column = 0)
frm_k2.grid(row = 3,column = 0)
frm_k3.grid(row = 4,column = 0)
frm_k4.grid(row = 5,column = 0)
frm_k5.grid(row = 0,column = 3)
frm_k6.grid(row = 1,column = 3)
frm_l.grid(row = 1,column = 0)
frm_r.grid(row = 1,column = 1)
frm_c.grid(row = 2,column = 1)
frm_c2.grid(row = 5,column = 1)
#----------Font--------------
ft1 = tkFont.Font(family='Fixdsys', size=16, weight=tkFont.BOLD)
ft2 = tkFont.Font(family='Fixdsys', size=14)
#Label(root, text='輸入區塊', font=ft1).grid()
#-----------------------Text---------------------------------
title = tk.Label(frm, text = "輸入區塊",font=ft1)
title.grid(row=0,column=0)
title2 = tk.Label(frm, text = "回測數據",font=ft1)
title2.grid(row=0,column=1)
title3 = tk.Label(frm_k, text = "選擇策略條件",font=ft1)
title3.grid(row=0,column=0,sticky='NW', pady = 5)
title4 = tk.Label(frm_k5, text = "虛擬交易/提醒功能",font=ft1)
title4.grid(row=0,column=0,sticky='NW', pady = 5)
stockCode = tk.Label(frm_l, text="輸入股票代碼:")#股票代碼
stockCode.grid(row=1,column=0,sticky='NW')
startTime = tk.Label(frm_l, text="輸入起始時間:")#股票日期
startTime.grid(row=2,column=0,sticky='NW')
endTime = tk.Label(frm_l, text='輸入結束時間')#股票日期
endTime.grid(row=3,column=0,sticky='NW')
start_money = tk.Label(frm_l, text="輸入初始金額:")#初始金額
start_money.grid(row=4,column=0,sticky='NW')
sProfit = tk.Label(frm_l, text="輸入停利條件:")#停利
sProfit.grid(row=5,column=0,sticky='NW')
sLoss = tk.Label(frm_l, text="輸入停損條件:")#停損
sLoss.grid(row=6,column=0,sticky='NW')

#-------------------------table------------------------------
table=ttk.Treeview(frm_r,show='headings')#表格
table["columns"]=("買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額","賣出獲利")
table.column("買/賣 狀態",width=100)   #表示列,不显示
table.column("交易日期",width=100)
table.column("股票價格",width=100)
table.column("股票庫存",width=100)
table.column("資金餘額",width=100)
table.column("賣出獲利",width=100)

table.heading("買/賣 狀態",text="買/賣 狀態")  #显示表头
table.heading("交易日期",text="交易日期")
table.heading("股票價格",text="股票價格")
table.heading("股票庫存",text="股票庫存")
table.heading("資金餘額",text="資金餘額")
table.heading("賣出獲利",text="賣出獲利")
table.grid()
#--------------------------textPanel-----------------------------------
#報酬率
pOfrate = tk.Label(frm_k4, text=' 報 酬 率 ',font=ft1)
pOfrate.grid(row =14, column = 0,sticky='NW', pady = 5)
#textPanel = tk.Text(frm_l,height=1, width=4)
#textPanel.grid(row =14,column=1,sticky='W')
#報酬率 1
ROP1 = tk.Label(frm_k4, text=' K值大小')
ROP1.grid(row =15, column = 0,sticky='NW')
textPanel1 = tk.Text(frm_k4,height=1, width=4)
textPanel1.grid(row =15, column = 1,sticky='NW')
ROP2 = tk.Label(frm_k4, text=' 連三反彈')
ROP2.grid(row =16, column = 0,sticky='NW')
textPanel2 = tk.Text(frm_k4,height=1, width=4)
textPanel2.grid(row =16, column = 1,sticky='NW')
ROP3 = tk.Label(frm_k4, text=' 黃金交叉')
ROP3.grid(row =17, column = 0,sticky='NW')
textPanel3 = tk.Text(frm_k4, height=1, width=4)
textPanel3.grid(row =17, column = 1,sticky='NW')
answer = tk.Label(frm_k4, text=' 比較結果:')
answer.grid(row =18, column = 0,sticky='NW')


#預設輸入
stock = tk.StringVar(value='2330.TW')
time = tk.StringVar(value='2018')
timeEnd = tk.StringVar(value='2019')
sMoney =tk.StringVar(value='1000000')
pEntry = tk.StringVar(value='5')
lEntry = tk.StringVar(value='5')
#----------------------------entry-----------------------------------------
stockCode_entry = tk.Entry(frm_l,textvariable=stock)
stockCode_entry.grid(row=1,column=1,sticky='W')
startTime_entry = tk.Entry(frm_l,textvariable=time)
startTime_entry.grid(row=2,column=1,sticky='W')
endTime_entry = tk.Entry(frm_l,textvariable=timeEnd)
endTime_entry.grid(row=3,column=1,sticky='W')
start_money_entry = tk.Entry(frm_l,textvariable=sMoney)
start_money_entry.grid(row=4,column=1,sticky='W')
sProfit_entry = tk.Entry(frm_l,textvariable=pEntry)
sProfit_entry.grid(row=5,column=1,sticky='W')
sLoss_entry = tk.Entry(frm_l,textvariable=lEntry)
sLoss_entry.grid(row=6,column=1,sticky='W')
#-------------------------Optionmenu-------------------------#
optionList = ["K值小於設定值進場，大於設定值出場","K值連續3天小於設定值後反彈進場","黃金交叉與死亡交叉"]
v = tk.StringVar()
v.set("選擇條件")
optionmenu = tk.OptionMenu(frm_k2, v, *optionList)#command=func(*optionList))
optionmenu.grid(row=1,column=0,sticky='W')
#en.pack()

#-------------------------Button-------------------------#
btn = tk.Button(frm_k3, text="單一回測",command=callfunc)
btn.grid(row = 0,column = 0,pady = 5, padx = 5)
btn2 = tk.Button(frm_c, text="清除資料", command=deletetable)
btn2.grid(row = 20,column = 5, padx = 10)
btn3 = tk.Button(frm_k6, text="虛擬交易平台",font=ft2, command=order)
btn3.grid(row = 0,column = 0, pady = 5)
btn4 = tk.Button(frm_k3, text="全部回測", command = Run3)
btn4.grid(row = 0,column = 1, pady = 5, padx = 5)
btn5 = tk.Button(frm_k6,text="指標自動提醒",font=ft2,command =callfunc2)
btn5.grid(row = 1,column = 0,pady =5)
btn6 = tk.Button(frm_k6,text="股價自動提醒",font=ft2,command =autoRemindPrice.priceRemind)
btn6.grid(row = 2,column = 0,pady =5)



win.mainloop()

2 1000000
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
2018-02-08 00:00:00 K值為: 18.28282828282828
2018-02-09 00:00:00 K值為: 16.829836829836825
2018-02-12 00:00:00 K值為: 17.76223776223776
2018-02-21 00:00:00 K值為: 26.666666666666668
買了一張股票，股價為: 242.5
[242.5]
剩餘資金為: 757500.0
擁有1張股票
2018-02-22 00:00:00
未達買賣點
2018-02-23 00:00:00
未達買賣點
2018-02-26 00:00:00
未達買賣點
2018-02-27 00:00:00
未達買賣點
2018-03-01 00:00:00
未達買賣點
2018-03-02 00:00:00
未達買賣點
2018-03-05 00:00:00
未達買賣點
2018-03-06 00:00:00
現在的股價= 250.0
條件: 現股>=最後一次購買股價+5 = 
最後一次購買股價 242.5
最後一次購買股價+5 247.5
已達停利標準
賣出1張股票
股價為: 250.0
擁有資金: 1007500.0
獲利: 7500.0
報酬率: 0.75
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
2018-05-02 00:00:00 K值為: 14.893617021276595
2018-05-03 00:00:00 K值為: 12.796793092815294
2018-05-04 00:00:00 K值為: 16.786154794942956
2018

買了一張股票，股價為: 239.5
[239.5]
剩餘資金為: 842000.0
擁有1張股票
2018-08-20 00:00:00
152
239.5
K值為: 13.333333333333355
買了一張股票，股價為: 239.5
[239.5, 239.5]
剩餘資金為: 602500.0
擁有2張股票
2018-08-21 00:00:00
153
241.0
尚未達到買賣點
2018-08-22 00:00:00
154
242.0
尚未達到買賣點
2018-08-23 00:00:00
155
244.5
現在的股價= 244.5
條件: 現股>=最後一次購買股價+10 = 
最後一次購買股價 239.5
最後一次購買股價+5 244.5
已達停利標準
賣出2張股票
股價為: 244.5
擁有資金: 1091500.0
獲利: 91500.0
報酬率: 9.15
2018-08-24 00:00:00
156
243.5
尚未達到買點
2018-08-27 00:00:00
157
245.0
尚未達到買點
2018-08-28 00:00:00
158
249.5
尚未達到買點
2018-08-29 00:00:00
159
259.0
尚未達到買點
2018-08-30 00:00:00
160
263.5
尚未達到買點
2018-08-31 00:00:00
161
256.0
尚未達到買點
2018-09-03 00:00:00
162
257.0
尚未達到買點
2018-09-04 00:00:00
163
257.5
尚未達到買點
2018-09-05 00:00:00
164
264.0
尚未達到買點
2018-09-06 00:00:00
165
261.0
尚未達到買點
2018-09-07 00:00:00
166
264.0
尚未達到買點
2018-09-10 00:00:00
167
264.5
尚未達到買點
2018-09-11 00:00:00
168
260.0
尚未達到買點
2018-09-12 00:00:00
169
260.5
尚未達到買點
2018-09-13 00:00:00
170
255.0
尚未達到買點
2018-09-14 00:00:00
171
261.0
尚未達到買點
2018-09-17 0